In [1]:
import numpy as np
import math
from fractions import Fraction

In [20]:
# do a pairwise check -- certainly not optimal, and doesn't help with part 2 :(
ast = {}
field = []
ast_ind = 0
with open("p10_input.txt","r") as f:
    for i, l in enumerate(f):
        r = []
        for j, c in enumerate(l.strip()):
            if c == "#":
                r.append(1)
                ast[ast_ind] = (i,j)
                ast_ind += 1
            else:
                r.append(0)
        field.append(r)
field = np.array(field)
n_ast = len(ast)
ast_graph = np.zeros((n_ast,n_ast))
ind_to_ast = -1*np.ones(field.shape,dtype=int)
for k,v in ast.items():
    ind_to_ast[v] = k
for i in range(n_ast):
    for j in range(i+1,n_ast):
        if ast_graph[i,j] == 1:
            continue
        dx = ast[j][0] - ast[i][0]
        dy = ast[j][1] - ast[i][1]
        xdir = np.sign(dx)
        ydir = np.sign(dy)
        if dy == 0: # vertical line, deal with separately
            ydiff = 0
            xdiff = 1
        else:
            frac = Fraction(dx,dy)
            xdiff = np.abs(frac.numerator)
            ydiff = np.abs(frac.denominator)
        pos = ast[i]
        while pos != ast[j]:
            pos = (pos[0] + xdir*xdiff,pos[1] + ydir*ydiff)
            if field[pos] == 1: # asteroid at current position
                ast_graph[i,ind_to_ast[pos]] = 1
                ast_graph[ind_to_ast[pos],i] = 1
                break
print("Part 1 answer: {}".format(int(ast_graph.sum(axis=0).max())))

Part 1 answer: 292


In [33]:
# another approach to part 1 that will also help with part 2, but even slower than my first approach :()
def positive_quadrant_rays(n_cols,n_rows):
    """
    Return an ordered list of the rays pointing strictly into
    the positive quadrant, in the format of Fractions,
    where the numerator is the vertical rise (hence negative diff in row)
    and the denominator is the rightward run (hence positive diff in col)
    Don't include vertical or horizontal.
    """
    rays = []
    for i in range(n_cols):
        for j in range(n_rows):
            if math.gcd(i+1,j+1) > 1: # we will deal with this fraction later?
                continue
            rays.append(Fraction(j+1,i+1))
    return sorted(rays,reverse=True)
field = []
with open("p10_input.txt","r") as f:
    for i, l in enumerate(f):
        r = []
        for j, c in enumerate(l.strip()):
            if c == "#":
                r.append(1)
            else:
                r.append(0)
        field.append(r)
field = np.array(field)
n_rows, n_cols = field.shape
qrays = generate_all_rays(n_cols,n_rows)
ast_ind = 0
ast_dict = {}
for r in range(n_rows):
    for c in range(n_cols):
        if field[r,c] == 0: # not an asteroid
            continue
            
        ast_dict[ast_ind] = [0,(r,c)]
        pos = (r,c)

        # test vertical up
        rdiff = -1
        cdiff = 0
        test_pos = (pos[0]+rdiff,pos[1]+cdiff)
        while test_pos[0] >= 0:
            if field[test_pos] == 1:
                ast_dict[ast_ind][0] += 1
                break
            test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

        # test positive quadrant rays
        for i in range(len(qrays)):
            ray = qrays[i]
            rdiff = -ray.numerator
            cdiff = ray.denominator
            test_pos = (pos[0]+rdiff,pos[1]+cdiff)
            while test_pos[0] >= 0 and test_pos[1] < n_cols:
                if field[test_pos] == 1:
                    ast_dict[ast_ind][0] += 1
                    break
                test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

        # test horizontal right
        rdiff = 0
        cdiff = 1
        test_pos = (pos[0]+rdiff,pos[1]+cdiff)
        while test_pos[1] < n_cols:
            if field[test_pos] == 1:
                ast_dict[ast_ind][0] += 1
                break
            test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

        # test lower right quadrant
        for i in range(len(qrays)):
            ray = qrays[-1-i]
            rdiff = ray.numerator
            cdiff = ray.denominator
            test_pos = (pos[0]+rdiff,pos[1]+cdiff)
            while test_pos[0] < n_rows and test_pos[1] < n_cols:
                if field[test_pos] == 1:
                    ast_dict[ast_ind][0] += 1
                    break
                test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

        # test vertical down
        rdiff = 1
        cdiff = 0
        test_pos = (pos[0]+rdiff,pos[1]+cdiff)
        while test_pos[0] < n_rows:
            if field[test_pos] == 1:
                ast_dict[ast_ind][0] += 1
                break
            test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

        # test lower left quadrant
        for i in range(len(qrays)):
            ray = qrays[i]
            rdiff = ray.numerator
            cdiff = -ray.denominator
            test_pos = (pos[0]+rdiff,pos[1]+cdiff)
            while test_pos[0] < n_rows and test_pos[1] >= 0:
                if field[test_pos] == 1:
                    ast_dict[ast_ind][0] += 1
                    break
                test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

        # test left horizontal
        rdiff = 0
        cdiff = -1
        test_pos = (pos[0]+rdiff,pos[1]+cdiff)
        while test_pos[1] >= 0:
            if field[test_pos] == 1:
                ast_dict[ast_ind][0] += 1
                break
            test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

        # test upper left quadrant
        for i in range(len(qrays)):
            ray = qrays[-1-i]
            rdiff = -ray.numerator
            cdiff = -ray.denominator
            test_pos = (pos[0]+rdiff,pos[1]+cdiff)
            while test_pos[0] >= 0 and test_pos[1] >= 0:
                if field[test_pos] == 1:
                    ast_dict[ast_ind][0] += 1
                    break
                test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

        ast_ind += 1
ast_counts = np.array([v[0] for v in ast_dict.values()])
print("Part 1 answer: {}".format(int(ast_counts.max())))

Part 1 answer: 292


In [34]:
# part 2 -- basically same structure as part 1, but delete instead of accumulate counts
destroyed = []
pos = ast_dict[ast_counts.argmax()][1]

while field.sum() > 1:
    # delete vertical up
    rdiff = -1
    cdiff = 0
    test_pos = (pos[0]+rdiff,pos[1]+cdiff)
    while test_pos[0] >= 0:
        if field[test_pos] == 1:
            field[test_pos] = 0
            destroyed.append(test_pos)
            break
        test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

    # delete positive quadrant rays
    for i in range(len(qrays)):
        ray = qrays[i]
        rdiff = -ray.numerator
        cdiff = ray.denominator
        test_pos = (pos[0]+rdiff,pos[1]+cdiff)
        while test_pos[0] >= 0 and test_pos[1] < n_cols:
            if field[test_pos] == 1:
                field[test_pos] = 0
                destroyed.append(test_pos)
                break
            test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

    # delete horizontal right
    rdiff = 0
    cdiff = 1
    test_pos = (pos[0]+rdiff,pos[1]+cdiff)
    while test_pos[1] < n_cols:
        if field[test_pos] == 1:
            field[test_pos] = 0 
            destroyed.append(test_pos)
            break
        test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

    # delete lower right quadrant
    for i in range(len(qrays)):
        ray = qrays[-1-i]
        rdiff = ray.numerator
        cdiff = ray.denominator
        test_pos = (pos[0]+rdiff,pos[1]+cdiff)
        while test_pos[0] < n_rows and test_pos[1] < n_cols:
            if field[test_pos] == 1:
                field[test_pos] = 0
                destroyed.append(test_pos)
                break
            test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

    # delete vertical down
    rdiff = 1
    cdiff = 0
    test_pos = (pos[0]+rdiff,pos[1]+cdiff)
    while test_pos[0] < n_rows:
        if field[test_pos] == 1:
            field[test_pos] = 0
            destroyed.append(test_pos)
            break
        test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

    # delete lower left quadrant
    for i in range(len(qrays)):
        ray = qrays[i]
        rdiff = ray.numerator
        cdiff = -ray.denominator
        test_pos = (pos[0]+rdiff,pos[1]+cdiff)
        while test_pos[0] < n_rows and test_pos[1] >= 0:
            if field[test_pos] == 1:
                field[test_pos] = 0
                destroyed.append(test_pos)
                break
            test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

    # delete left horizontal
    rdiff = 0
    cdiff = -1
    test_pos = (pos[0]+rdiff,pos[1]+cdiff)
    while test_pos[1] >= 0:
        if field[test_pos] == 1:
            field[test_pos] = 0
            destroyed.append(test_pos)
            break
        test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)

    # delete upper left quadrant
    for i in range(len(qrays)):
        ray = qrays[-1-i]
        rdiff = -ray.numerator
        cdiff = -ray.denominator
        test_pos = (pos[0]+rdiff,pos[1]+cdiff)
        while test_pos[0] >= 0 and test_pos[1] >= 0:
            if field[test_pos] == 1:
                field[test_pos] = 0
                destroyed.append(test_pos)
                break
            test_pos = (test_pos[0]+rdiff, test_pos[1]+cdiff)
dest_pos = destroyed[199]

print("Part 2 answer: {}".format(100*dest_pos[1] + dest_pos[0]))

Part 2 answer: 317
